In [20]:
import xml.etree.ElementTree as ET
import pandas as pd
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

df=pd.DataFrame(columns=["id","date", "application_reference", "title","abstract","ipcr_classifications", "claims", "embedding"])
with open("xmls/temp/temp.xml") as input:
    n_line=0
    #temp_file = open('xmls/temp/temp.xml','w')
    while True:
        line = input.readline()
        if not line: 
            break
        #if line.startswith('<?xml') and n_line!=0:
            #temp_file.close()
        tree = ET.parse('xmls/temp/temp.xml')
        us_patent_grant = tree.getroot()
        title=None
        ab=None
        application_ref={}
        for application_reference in us_patent_grant.iter("application-reference"):
            application_ref["application_id"]=application_reference.find("document-id").find("doc-number").text
            application_ref["application_date"]=application_reference.find("document-id").find("date").text
        for us_bibliographic_data_grant in us_patent_grant.iter("us-bibliographic-data-grant"):
            for publication_reference in us_patent_grant.iter("publication-reference"):
                publication_id=publication_reference.find("document-id").find("doc-number").text
                publication_date=publication_reference.find("document-id").find("date").text
            title=us_bibliographic_data_grant.find("invention-title").text
            classifications_ipcr=list()
            classifications_cpc=list()
            classification_locarno=list()
            classification_national=list()
            for classifications in us_bibliographic_data_grant.iter("classifications-ipcr"):
                for classification in classifications.iter("classification-ipcr"):
                    classifications_ipcr.append({
                        "section": classification.find("section").text,
                        "class": classification.find("class").text,
                        "subclass": classification.find("subclass").text,
                        "main-group": classification.find("main-group").text
                        })
            for classification in us_bibliographic_data_grant.iter("classifications-locarno"):
                classifications_ipcr.append({
                    "edition": classification.find("edition").text,
                    "main-classification": classification.find("main-classification").text,
                    "further-classification": classification.find("further-classification").text,
                    "text": classification.find("text").text
                    })
        for abstract in us_patent_grant.iter("abstract"):
            ab="".join(abstract.itertext())
        claims=list()
        for cls in us_patent_grant.iter("claims"):
            for claim in cls.iter("claim"):
                cl="".join(claim.itertext())
                claims.append(cl)    
    if ab!=None :
        emb=model.encode(ab)
        df.loc[len(df)]=[ publication_id, publication_date, application_ref, title, ab, classifications_ipcr, claims, emb]
        #   temp_file = open('xmls/temp/temp.xml','w')
        #temp_file.write(line)   
        n_line+=1
df.to_csv("xmls/temp/temp.csv")